## 0. loading packages

In [4]:
import os, sys
import codecs
import gensim
import pickle
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm

## 1. loading data

In [25]:
data_path = os.path.join(os.environ['HOME'], 'data')
config_path = os.path.join(os.environ['HOME'], 'config')

In [30]:
con = sqlite3.connect(os.path.join(data_path, 'database.sqlite'))
cursor = con.cursor()

In [31]:
table = pd.read_sql_query("SELECT * from sqlite_master", con)

In [32]:
table

,type,name,tbl_name,rootpage,sql


In [ ]:
cursor.execute('''SELECT name, email, phone FROM users''')
user1 = cursor.fetchone() #retrieve the first row
print(user1[0]) #Print the first column retrieved(user's name)
all_rows = cursor.fetchall()